In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

In [12]:
class Inception(nn.Module):
    # c1- c4 are number of output channels for each branch
    def __init__(self, c1, c2, c3, c4, **kwargs):
        super().__init__(**kwargs)
        # Branch 1
        self.b1_1 = nn.LazyConv2d(c1,kernel_size=1)
        # Branch 2
        self.b2_1 = nn.LazyConv2d(c2[0], kernel_size=1)
        self.b2_2 = nn.LazyConv2d(c2[1], kernel_size=3, padding=1)
        # Branch 3
        self.b3_1 = nn.LazyConv2d(c3[0], kernel_size=1)
        self.b3_2 = nn.LazyConv2d(c3[1], kernel_size=5, padding=2)
        # Branch 4
        self.b4_1 = nn.MaxPool2d(kernel_size=3, padding=1,stride=1)
        self.b4_2 = nn.LazyConv2d(c4, kernel_size=1)
    
    def forward(self, X):
        b1 = F.relu(self.b1_1(X))
        b2 = F.relu(self.b2_2(F.relu(self.b2_1(X))))
        b3 = F.relu(self.b3_2(F.relu(self.b3_1(X))))
        b4 = F.relu(self.b4_2(F.relu(self.b4_1(X))))
        return torch.cat((b1, b2, b3 , b4), dim=1)

In [13]:
class GoogleNet(d2l.Classifier):
    def b1(selfs):
        return nn.Sequential(
            nn.LazyConv2d(64, kernel_size=7, stride=2, padding=3),
            nn.ReLU(), nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

In [14]:
@d2l.add_to_class(GoogleNet)
def b2(self):
    return nn.Sequential(
        nn.LazyConv2d(64, kernel_size=1), nn.ReLU(),
        nn.LazyConv2d(192, kernel_size=3, padding=1), nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
    )

In [15]:
@d2l.add_to_class(GoogleNet)
def b3(self):
    return nn.Sequential(Inception(c1=64, c2=(96, 128), c3=(16, 32), c4=32),
                         Inception(c1=128, c2=(128, 192), c3=(32, 96), c4=64),
                         nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
                         )

In [16]:
@d2l.add_to_class(GoogleNet)
def b4(self):
    return nn.Sequential(Inception(192, (96, 208), (16, 48), 64),
                         Inception(160, (112, 224), (24, 64), 64),
                         Inception(128, (128, 256), (24, 64), 64),
                         Inception(112, (144, 288), (32, 64), 64),
                         Inception(256, (160, 320), (32, 128), 128),
                         nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [17]:
@d2l.add_to_class(GoogleNet)
def b5(self):
    return nn.Sequential(Inception(256, (160, 320), (32, 128), 128),
                         Inception(384, (192, 384), (48, 128), 128),
                         nn.AdaptiveAvgPool2d((1,1)), nn.Flatten())

In [18]:
@d2l.add_to_class(GoogleNet)
def __init__(self, lr=0.1, num_classes=10):
    super().__init__()
    self.save_hyperparameters()
    self.net = nn.Sequential(self.b1(), self.b2(), self.b3(). self.b4(), 
                             self.b5(), nn.LazyLinear(num_classes))
    self.net.apply(d2l.init_cnn)